In [ ]:
# When R2 is negative, Model is performing worse than the mena.

# Need to Add ability to filter ML Models Based on Dataset size.
# Hard Code Small, Medium Large, 
# Add new columns and helpful hitns to Datasheet

In [ ]:
## As Part of ML Process Need to Develop a specific plan, including Reviewing Documentation, Updating DOcumentation 

In [1]:
import numpy as np

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import time
import sys

sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

from MLPipeline import MLPipelineSample
#CalculateRegressionPerformance,SKLearnModelList,apply_scaling
#from sklearn.model_selection import train_test_split

from DataSets import iris_df,diabetes_df

df = iris_df.copy()
df1 = diabetes_df.copy()

results_df = MLPipelineSample(df1,
                              scaler='normal',
                              target_column='Target',
                              sample_override=1,
                              run_all_size_models=1,
                              test_size=0.2)



In [ ]:

from DataSets import MNIST_SKLEARN
from MLPipeline import MLPipelineSample

df,z = MNIST_SKLEARN(normalize=False,flatten=True,return_value='df')

results_df = MLPipelineSample(df,
                              scaler='normal',
                              ml_model_type='classifier',
                              target_column='Target',
                              test_size=0.2)
results_df


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import time
import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

from DataSets import MNIST_SKLEARN
from MLPipeline import SKLearnModelList,apply_scaling,ClassificationMetrics

df,z = MNIST_SKLEARN(normalize=False,flatten=True,return_value='df')

def MLPipeline(df, 
               scaler,
               ml_model_type='regressor',
               target_column='Target',
               sample_override=0,
               test_size=0.2):
    """
    Runs multiple scikit-learn estimators with MLflow tracking.

    Args:
        df (DataFrame): Input dataset.
        project_name (str): MLflow experiment name.
        scaler (str): One of None, 'normal', or 'standard'.
        ml_model_type (str): 'classifier', 'regressor', 'cluster', 'transformer'
        target_column (str): Name of the target column.
        test_size (float): Proportion of data used for testing.

    Returns:
        pd.DataFrame: Summary of model performance.
    """

    # Get model list (you must have this function already working)
    sklearn_models_df = SKLearnModelList()

    if len(df)<5000*(1+test_size):
        model_list = sklearn_models_df[(sklearn_models_df['Dataset Size'].str.contains('small',case=False))&(sklearn_models_df['Estimator Type'].str.contains(ml_model_type,case=False))]
    elif len(df)<100000*(1+test_size):
        model_list = sklearn_models_df[(sklearn_models_df['Dataset Size'].str.contains('medium',case=False))&(sklearn_models_df['Estimator Type'].str.contains(ml_model_type,case=False))]
    elif len(df)>100000*(1+test_size):
        model_list = sklearn_models_df[(sklearn_models_df['Dataset Size'].str.contains('large',case=False))&(sklearn_models_df['Estimator Type'].str.contains(ml_model_type,case=False))]

    if len(df)>5000 & sample_override==0:
        df = df.sample(frac=.15).copy()
        
    # Prepare data
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Apply scaler
    X_train, X_test, _ = apply_scaling(X_train, X_test, scaler=scaler)

    results_df = pd.DataFrame()

    for _, row in model_list.iterrows():
        name = row['Model Name']
        estimator_class = row['Estimator Class']
        print(f'Generating Predicition for {name}, {estimator_class}')
               
        try:
            start_time = time.time()
            model = estimator_class()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Calculate AUC Score Outside of Classification as it requires information not being passed.
            if hasattr(model, "predict_proba"):
                y_proba = model.predict_proba(X_test)
                AUC_Score = roc_auc_score(y_test, y_proba, multi_class="ovr")
                print(AUC_Score)
            else:
                AUC_Score = 0            
            
            binary_df = pd.concat([pd.DataFrame(y_pred,columns=['PREDICTION']),pd.DataFrame(y_test).rename(columns={target_column:'ACTUAL'}).reset_index(drop=True)],axis=1)            
            model_perfom_stats =  ClassificationMetrics(binary_df,AUC_Score=AUC_Score)
            model_perfom_stats['Model'] = name
            model_perfom_stats['Scaler'] = scaler

            end_time = time.time()
            elapsed = end_time - start_time
            model_perfom_stats['Time'] = elapsed
            results_df = pd.concat([results_df,model_perfom_stats])
            print(f'{name} Successfully Completed in {elapsed}.')

        except Exception as e:
            print(f"Model Generation and Results Failed: {e}\n")
    return results_df
        
results_df = MLPipeline(df,
               scaler='normal',
               ml_model_type='classifier',
               target_column='Target',
               test_size=0.2)
